## Load the private key from the file

In [54]:
import os
import yaml
import base64
import pickle
import json
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.fernet import Fernet

In [8]:
script_dir = os.getcwd()
parent_dir = os.path.dirname(script_dir)
private_params_path = os.path.join(parent_dir, 'private_params.yml')

In [157]:
with open(private_params_path, 'r') as file:
    private_params = yaml.safe_load(file)

In [158]:
private_params

{'testing_data_pwd': '151678', 'testing_data_salt': 'finpricing'}

## Utility functions

In [152]:
def generate_key_from_password(password, salt_str):
    if not isinstance(password, str):
        raise TypeError('Password must be a string')
    if not isinstance(salt_str, str):
        raise TypeError('Salt must be a string.')
    salt = salt_str.encode()
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
    return key

In [125]:
def encrypt_data_with_key(data: str | bytes, key: bytes):
    if not isinstance(data, str) and not isinstance(data, bytes):
        raise TypeError('Data must be a string / bytes')
    if isinstance(data, str):
        data = data.encode()
    cipher_suite = Fernet(key)
    encrypted_data = cipher_suite.encrypt(data)
    return encrypted_data

In [126]:
def store_encrypted_data(encrypted_data, file_path):
    if not isinstance(encrypted_data, bytes):
        raise TypeError('Encrypted data must be bytes')
    if not isinstance(file_path, str):
        raise TypeError('File path must be a string')
    with open(file_path, 'wb') as file:
        pickle.dump(encrypted_data, file)

In [127]:
def load_and_decrypt_data(file_path, key):
    if not isinstance(file_path, str):
        raise TypeError('File path must be a string')
    if not isinstance(key, bytes):
        raise TypeError('Key must be bytes')
    with open(file_path, 'rb') as file:
        encrypted_data = pickle.load(file)
    cipher_suite = Fernet(key)
    decrypted_data = cipher_suite.decrypt(encrypted_data)
    assert isinstance(decrypted_data, bytes), "Decrypted data must be bytes, and then loaded by pickle"
    return pickle.loads(decrypted_data)

In [164]:
key = generate_key_from_password(private_params['testing_data_pwd'], private_params['testing_data_salt'])

## Encrypt the message

In [179]:
testing_data_path = os.path.join(parent_dir, 'tests/testing_data')
file_path = os.path.join(testing_data_path, 'bondcurve_portfolio.json')

In [180]:
with open(file_path, 'rb') as file:
    discount_curve = json.load(file)

In [181]:
pickled_data = pickle.dumps(discount_curve)

In [182]:
encrypted_data = encrypt_data_with_key(pickled_data, key)

In [183]:
store_encrypted_data(encrypted_data, os.path.join(testing_data_path, 'bondcurve_portfolio_20231009.pkl'))

In [184]:
loaded = load_and_decrypt_data(os.path.join(testing_data_path, 'bondcurve_portfolio_20231009.pkl'), key)